# Статистические выбросы
Определим наименее характерные данные и удалим их. Построим классификацию по KMeans и оценим ее точность относительно нефильтрованных данных.
![Критерий Смирнова-Граббса](smirnov.png)
### Данные
* https://video.ittensive.com/machine-learning/hacktherealty/exposition_train.basic.csv.gz

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from outliers import smirnov_grubbs as grubbs

### Загрузка данных

In [2]:
train_data = pd.read_csv('https://video.ittensive.com/machine-learning/hacktherealty/exposition_train.basic.csv.gz')
train_data.head()

,total_area,ceiling_height,rooms,living_area,price,day_mean,doy_108,price_locality_name_median,target
0,105.000000,3.00,3,50.000000,95000,2.456912,0,2.261905,1
1,40.000000,3.00,1,19.200001,25000,3.028689,0,1.000000,2
2,37.599998,2.64,0,19.000000,26000,3.091993,0,0.619048,2
3,80.000000,3.00,3,49.000000,35000,3.101010,0,1.250000,2
4,100.000000,3.00,3,49.000000,80000,2.495468,0,1.904762,3


### Нормализация данных
Приведение всех значений к отрезку [0;1], удалим из данных target

In [3]:
train_data_an = pd.DataFrame(StandardScaler().fit_transform(train_data[train_data.columns[:-1]]))

In [4]:
train_data_an.head()

,0,1,2,3,4,5,6,7
0,1.709793,1.115052,1.469889,1.025395,0.378181,-1.607090,-0.108831,0.278301
1,-0.483202,1.115052,-0.861151,-0.679471,-0.217544,0.337784,-0.108831,-0.132179
2,-0.564174,-0.645106,-2.026671,-0.690542,-0.209033,0.553112,-0.108831,-0.256098
3,0.866333,1.115052,1.469889,0.970042,-0.132440,0.583783,-0.108831,-0.050858
4,1.541101,1.115052,1.469889,0.970042,0.250526,-1.475943,-0.108831,0.162128


### Тест Смирнова-Граббса
Найдем нехарактерные данные за границами нормальности (уровень 99,9%). Отдельно по каждому атрибуту - сохраним индексы в наборе данных

In [5]:
anomalies = []
for column in train_data_an.columns:
    print ("Обработка столбца", column)
    anomaly = grubbs.two_sided_test_indices(np.array(train_data_an[column]), alpha=.1)
    anomalies.extend(anomaly)

Обработка столбца 0
Обработка столбца 1
Обработка столбца 2
Обработка столбца 3
Обработка столбца 4
Обработка столбца 5
Обработка столбца 6
Обработка столбца 7


Отфильтруем аномальные данные

In [6]:
train_data_filtered = train_data_an[~train_data_an.index.isin(anomalies)]
print ("Процент аномалий:", round(100*len(anomalies)/len(train_data_an),2), "%")

Процент аномалий: 7.37 %


### Сравним K-средних
Построим две модели: на данных без фильтрации и отфильтрованных

In [7]:
kmeans_base = KMeans(n_clusters=100, random_state=0, max_iter=100, n_init=10).fit(train_data_an)

In [8]:
kmeans_an = KMeans(n_clusters=100, random_state=0, max_iter=100, n_init=10).fit(train_data_filtered)

In [9]:
target_cluster_base = kmeans_base.predict(train_data_an)
target_cluster_an = kmeans_an.predict(train_data_an)

In [10]:
train_data_an["target_cluster_base"] = target_cluster_base
train_data_an["target_cluster_an"] = target_cluster_an

### Оценка точности предсказания
Присвоим среднее значение по кластеру

In [11]:
train_data_an["target"] = train_data["target"]

In [12]:
target_cluster_base = np.round(train_data_an.groupby("target_cluster_base").mean()["target"])
train_data_an["target_pred_base"] = train_data_an["target_cluster_base"].apply(lambda x: target_cluster_base[x])

In [13]:
target_cluster_an = np.round(train_data_an.groupby("target_cluster_an").mean()["target"])
train_data_an["target_pred_an"] = train_data_an["target_cluster_an"].apply(lambda x: target_cluster_an[x])

In [14]:
print ("100 средних: ", np.exp(np.abs(train_data_an["target"] - train_data_an["target_pred_base"])).sum() / len(train_data_an))

100 средних:  4.186247152543807


In [15]:
print ("Фильтрация аномалий: ", np.exp(np.abs(train_data_an["target"] - train_data_an["target_pred_an"])).sum() / len(train_data_an))

Фильтрация аномалий:  4.162560430231132
